In [4]:
import sys, os
import numpy as np

sys.path.append(os.getcwd())
from config import *
from utils import *
from model import *

## Provide test Details

In [5]:
test_name = "SampleTest2"
Day = "Monday"
Date = "10_2_2021"
n_epochs="50"
n_images="25"
articulator="Tongue" #Airway, Tongue, Velum

model_path = output_file_path(test_name,n_epochs,n_images,Day,Date, articulator)

## Model Architecture

In [6]:
import skimage.io as io
import skimage.transform as trans
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler



def unet(pretrained_weights=None, input_size=target_input_size):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)
    
    model = Model(inputs=inputs, outputs=conv10)
    
    if(pretrained_weights):
        model.load_weights(pretrained_weights)
        
    return model

## Model Traning

In [13]:
model = unet()
myGenerator = trainGenerator(8,target_data_root  ,'images' ,'mask', data_gen_args, target_img_size)
model.compile(optimizer = Adam(lr = 1e-4), loss = dice_coef, metrics = ['accuracy'])
model_checkpoint = ModelCheckpoint(model_path, monitor='loss',verbose=1, save_best_only=True)
model.fit(myGenerator,steps_per_epoch=5,epochs=5,callbacks=[model_checkpoint])

Found 20 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Epoch 1/5
5/5 [==============================] - 64s 12s/step - loss: 0.1712 - accuracy: 0.8287

Epoch 00001: loss improved from inf to 0.17116, saving model to Output/Models/Tongue/SampleTest2_50_25_Monday_10_2_2021.hdf5
Epoch 2/5
5/5 [==============================] - 54s 12s/step - loss: 0.1716 - accuracy: 0.8283

Epoch 00002: loss did not improve from 0.17116
Epoch 3/5
5/5 [==============================] - 55s 10s/step - loss: 0.1739 - accuracy: 0.8260

Epoch 00003: loss did not improve from 0.17116
Epoch 4/5
5/5 [==============================] - 60s 12s/step - loss: 0.1746 - accuracy: 0.8253

Epoch 00004: loss did not improve from 0.17116
Epoch 5/5
5/5 [==============================] - 54s 12s/step - loss: 0.1717 - accuracy: 0.8282

Epoch 00005: loss did not improve from 0.17116


In [14]:
import tensorflow as tf
testGene = testGenerator("data/Airway/images")
trainedModel = tf.keras.models.load_model(model_path, compile=False)
results = trainedModel.predict_generator(testGene,8,verbose=1)
saveResult("Output/Images/Tongue",results)

8/8 [==============================] - 4s 470ms/step


Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


[[[0.48384634]
  [0.44603765]
  [0.4625475 ]
  ...
  [0.42419708]
  [0.42483902]
  [0.47119665]]

 [[0.4765767 ]
  [0.43472177]
  [0.4567109 ]
  ...
  [0.42560738]
  [0.43674734]
  [0.4718485 ]]

 [[0.4728362 ]
  [0.43838555]
  [0.45161378]
  ...
  [0.405721  ]
  [0.42575952]
  [0.4488662 ]]

 ...

 [[0.4794389 ]
  [0.46546707]
  [0.4662515 ]
  ...
  [0.41025376]
  [0.43496966]
  [0.42673534]]

 [[0.47961628]
  [0.46571988]
  [0.46107644]
  ...
  [0.42673475]
  [0.44971198]
  [0.4462002 ]]

 [[0.48527414]
  [0.46365103]
  [0.45873734]
  ...
  [0.40921217]
  [0.44131097]
  [0.44385004]]]
[[[0.48469502]
  [0.44580567]
  [0.45359236]
  ...
  [0.36050394]
  [0.36884332]
  [0.45286494]]

 [[0.47629246]
  [0.43141693]
  [0.4457189 ]
  ...
  [0.3633325 ]
  [0.40044108]
  [0.47745854]]

 [[0.47688094]
  [0.44071624]
  [0.44395113]
  ...
  [0.32251197]
  [0.3447685 ]
  [0.41781914]]

 ...

 [[0.47671902]
  [0.46133536]
  [0.4626694 ]
  ...
  [0.4350981 ]
  [0.45793065]
  [0.45326936]]

 [[0.478

Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


[[[0.4906923 ]
  [0.47123858]
  [0.47405997]
  ...
  [0.4559905 ]
  [0.4548927 ]
  [0.48271117]]

 [[0.4850549 ]
  [0.4636867 ]
  [0.47040585]
  ...
  [0.45637822]
  [0.4639045 ]
  [0.48398808]]

 [[0.4812584 ]
  [0.46213865]
  [0.46768555]
  ...
  [0.44074437]
  [0.45350367]
  [0.4700775 ]]

 ...

 [[0.48874626]
  [0.47804752]
  [0.4784544 ]
  ...
  [0.4540647 ]
  [0.4744319 ]
  [0.46617833]]

 [[0.4877016 ]
  [0.4798819 ]
  [0.47867268]
  ...
  [0.4631208 ]
  [0.47560516]
  [0.47141612]]

 [[0.4899631 ]
  [0.47599685]
  [0.4739724 ]
  ...
  [0.4550664 ]
  [0.47233266]
  [0.47473451]]]
[[[0.48718485]
  [0.43965736]
  [0.44562203]
  ...
  [0.4176105 ]
  [0.41759482]
  [0.47019178]]

 [[0.47139394]
  [0.41527084]
  [0.42676544]
  ...
  [0.42176557]
  [0.438267  ]
  [0.4830048 ]]

 [[0.47059816]
  [0.42411575]
  [0.42498285]
  ...
  [0.39712998]
  [0.42040554]
  [0.45238242]]

 ...

 [[0.47333053]
  [0.45672405]
  [0.45545235]
  ...
  [0.4347868 ]
  [0.46227294]
  [0.4566974 ]]

 [[0.474